<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/%EA%B3%BC%EC%A0%9C/json_to_text%2C_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 샘플파일
!gdown 1rswWV67Idv8OXI1qQ7o5aSSWvP_uA5MW
# 맑음고딕 폰트
!gdown 1G4jAcJLy_1PC9YQR36t7jMapj9da4vPd

In [2]:
import zipfile
import os
import json
from PIL import Image, ImageDraw, ImageFont

In [3]:
!mkdir /content/sample
%cd /content/sample

/content/sample


In [ ]:
!unzip /content/sample.zip

In [5]:
!mkdir /content/text
!mkdir /content/jpg

/content/text


In [ ]:
foilderlist = os.listdir('/content/sample')
for i in foilderlist:
    per_path = f'/content/sample/{i}/20per'
    sent_path = f'/content/sample/{i}/2~3sent'
    per_list = os.listdir(per_path)
    sent_list = os.listdir(sent_path)
    for j in per_list:

        with open(f'{per_path}/{j}', 'r', encoding='utf-8') as file:
            text = json.load(file)
            passage =f"  {text['Meta(Refine)']['passage']}"

        # Save the extracted passage to the 'test' directory
            with open(f'/content/text/{j[:-4]}.txt', 'w', encoding='utf-8') as f:
                f.write(passage)

        # Create an image with the text
        image_width = 800  # Adjust as needed
        font_size = 18
        font_path = "/content/malgun.ttf"  # Replace with your font path

        # Load a Unicode font
        font = ImageFont.truetype(font_path, font_size)

        # Calculate image height and initialize variables
        lines = []
        line = ""
        left_margin = 50
        right_margin = 50
        for word in passage.split():
            # Check if adding the current word exceeds the available width
            if font.getsize(line + " " + word)[0] <= (image_width - left_margin - right_margin):
                line += " " + word
            else:
                lines.append(line)
                line = word

        lines.append(line)  # Append the last line

        # Calculate image height based on the number of lines
        vertical_space = 10
        top_margin = 50
        bottom_margin = 50
        line_height = font.getsize('A')[1]  # Height of a line of text
        image_height = line_height * len(lines) + (len(lines) - 1) * vertical_space + top_margin + bottom_margin

        # Create a new image with margins
        image = Image.new('RGB', (image_width, image_height), color='white')
        draw = ImageDraw.Draw(image)

        # Draw the text on the image with margins
        y_position = top_margin  # Adjust the starting y position
        text_color = (0, 0, 0)  # Black color

        for line in lines:
            x_position = left_margin  # Adjust the starting x position
            draw.text((x_position, y_position), line, fill=text_color, font=font)
            y_position += line_height + vertical_space

        # Save the image as JPG
        image.save(f'/content/jpg/{j[:-4]}.jpg')

<ipython-input-6-4ee7ab1f3460>:32: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  if font.getsize(line + " " + word)[0] <= (image_width - left_margin - right_margin):
<ipython-input-6-4ee7ab1f3460>:44: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  line_height = font.getsize('A')[1]  # Height of a line of text


### dount 실험

In [ ]:
!git clone https://github.com/clovaai/donut.git
%cd donut

In [ ]:
!pip install transformers==4.25.1
!pip install pytorch-lightning==1.6.4
!pip install timm==0.5.4
!pip install gradio
!pip install donut-python

#### 이미지 넣고 실험

In [ ]:
import torch
print ( "CUDA 사용 가능:" , torch.cuda.is_available())
!nvcc --version
# 이제 관련 파일과 라이브러리를 다운로드할 수 있습니다. 다음 코드 줄은 donut 라이브러리를 포함하여 모든 종속성을 설치해야 합니다
#pip install donut-python.

!git clone https://github.com/clovaai/donut.git
!cd donut && pip install .
# Making an inference using the CORD fine-tuned model
# First, we’ll demonstrate basic usage of the model.

In [ ]:
import torch
from donut import DonutModel
from PIL import Image

model = DonutModel.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")
if torch.cuda.is_available():
    model.half()
    device = torch.device("cuda")
    model.to(device)
else:
    model.encoder.to(torch.bfloat16)
model.eval()
image = Image.open("/content/test.PNG")
image.convert("RGB")
output = model.inference(image=image, prompt="<s_cord-v2>")
output

#### 훈련

In [ ]:
# train_dataset
from torchvision import transforms

class CustomDataset(Dataset):
    def __init__(self, image_folder, text_folder, transform=None):
        self.image_folder = image_folder
        self.text_folder = text_folder
        self.image_names = os.listdir(image_folder)
        self.text_names = os.listdir(text_folder)
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        image_name = self.image_names[idx]
        text_name = image_name.replace(".jpg", ".txt")  # Assuming TXT files have matching names

        image_path = os.path.join(self.image_folder, image_name)
        text_path = os.path.join(self.text_folder, text_name)

        image = Image.open(image_path).convert("RGB")

        # Apply transformations to convert the PIL image to a PyTorch tensor
        if self.transform:
            image = self.transform(image)

        with open(text_path, 'r') as f:
            text_data = f.read()

        return {'images': image, 'targets': text_data}

In [ ]:
# Define dataset paths and other settings
image_folder = '/content/jpg'
text_folder = '/content/text'
batch_size = 16
learning_rate = 1e-3
num_epochs = 10
desired_height = 800
desired_width = 1200

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((desired_height, desired_width)),  # Resize images to a common size
    transforms.ToTensor(),  # Converts PIL image to a PyTorch tensor
    # Add any other transformations you need here
])

# Create the custom dataset with the defined transformations
train_dataset = CustomDataset(image_folder, text_folder, transform=transform)

# Create DataLoader for batching
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
import torch
from donut import DonutModel
from torch.utils.data import DataLoader
from PIL import Image

# 데이터셋과 DataLoader를 여기에 정의하세요
# ...

# CPU 또는 GPU에서 훈련할 device를 정의합니다
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 이미 정의하고 준비한 train_dataset을 가정합니다
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# DonutModel 및 다른 훈련 설정을 정의합니다
model = DonutModel.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")
model.to(device)  # 모델을 지정한 device로 이동합니다
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.MSELoss()  # 적절한 손실 함수를 사용하세요

num_epochs = 10

# 훈련 루프
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch in train_data_loader:
        optimizer.zero_grad()

        # 배치가 이미지와 다른 필요한 입력을 포함한다고 가정합니다
        images = batch['images'].to(device)  # 이미지를 지정한 device로 이동합니다
        targets = batch['targets']  # 타겟이 텍스트 데이터라고 가정합니다

        # 텍스트 데이터를 전처리하여 'decoder_input_ids' 및 'decoder_labels'를 생성합니다
        tokenizer = ...  # 토크나이저를 초기화합니다
        inputs = tokenizer(targets, return_tensors='pt', padding='max_length', truncation=True, max_length=desired_max_length)

        # 순방향 패스
        outputs = model(encoder_input_ids=inputs['input_ids'].to(device),
                        decoder_input_ids=inputs['input_ids'].to(device),  # 이를 적절하게 수정해야 합니다
                        decoder_labels=inputs['input_ids'].to(device))  # 이를 적절하게 수정해야 합니다

        # 손실 계산
        loss = criterion(outputs, targets)
        total_loss += loss.item()

        # 역전파 및 최적화
        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_data_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}")